# RTF model for predicting racket type using datasets P1, P2, and P3, based on **peaks** - Vibration

### Model Description

This notebook implements a RTF model to predict the age of a racket (RV, RR, RB) based on sound features extracted from vibration datasets. The workflow involves reading `.csv` files, **extracting peaks** using FFT, and training the model using these features.

### Import libraries

In [1]:
import os
import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.io import wavfile
from scipy.fft import fft
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks
import librosa
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
# import openpyxl

### Import Tools Functions

In [2]:
import sys
# Reach the project root
notebook_path = os.path.abspath('')
project_root = os.path.abspath(os.path.join(notebook_path, '../../../'))
functions_path = os.path.join(project_root, 'Functions')

# Add Functions folder
if functions_path not in sys.path:
    sys.path.append(functions_path)

Energy

In [3]:
from extractPeakFromSignal import extractPeakFromSignal

# Main

In [4]:
import ast

# Read the CSV file
df_combined = pd.read_csv("../../../Data/Vibration/All_Data_combinedv3.csv")


df_combined.head()

,Raw Signal Ch0,Spectrum,freqs,File Name,Position,Racket Type,Age
0,[-0.01573961 -0.02470204 -0.0184555 ... -6.36...,"[5415.435319490299, 3187.673624625387, 2567.48...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_1.csv,C,RB,1
1,[ 0.10185837 0.10090781 0.08882211 ... -6.35...,"[3574.0039456266313, 2253.172677296551, 2189.4...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_2.csv,C,RB,1
2,[-0.02701054 -0.01506063 -0.02510942 ... -6.09...,"[4336.317505980091, 2636.183071505374, 2297.29...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_C_3.csv,C,RB,1
3,[-0.05687327 -0.06352719 -0.05049093 ... -9.29...,"[7218.047965980523, 4371.60331681831, 3234.473...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_S_1.csv,S,RB,1
4,[-0.08674804 -0.04994775 -0.05334261 ... -8.81...,"[6351.72555524914, 3431.4594161592727, 3006.71...","[158.33333333333331, 166.66666666666663, 174.9...",P1_RB_1_S_2.csv,S,RB,1


In [5]:
# Compute the feature
freqs = np.array(ast.literal_eval(df_combined["freqs"].iloc[0]))  # Convert string to numpy array

peaksAmplitude_list=[]
peaksFrequencies_list=[]

for spectrum in df_combined["Spectrum"]:
    spectrum_list = np.array(ast.literal_eval(spectrum))  
    peaksHZ, peaksAmplitude= extractPeakFromSignal(spectrum_list, smoothing=1,num_peaks=10)
    peaksAmplitude_list.append(peaksAmplitude["peak_heights"])
    peaksFrequencies_list.append(peaksHZ)



df_combined["peaksAmplitude"]=peaksAmplitude_list
df_combined["peaksHZ"]=peaksFrequencies_list

In [6]:
# Determine the minimum number of samples for each age group
min_samples_per_age = df_combined['Racket Type'].value_counts().min()

# Create a new DataFrame with equal samples for each age group
df_sorted_homogeneous = (
    df_combined.groupby('Racket Type')
    .apply(lambda x: x.sample(n=min_samples_per_age, random_state=42))
    .reset_index(drop=True)
)

# df_sorted_homogeneous

# Verify the distribution
print(df_sorted_homogeneous['Racket Type'].value_counts())




# Homogenize the data by ensuring all samples have the same length using mean values instead of 0
max_length = max(len(peaks) for peaks in df_sorted_homogeneous["peaksAmplitude"])
X = np.array([np.pad(peaks, (0, max_length - len(peaks)), 'constant', constant_values=np.mean(peaks)) for peaks in df_sorted_homogeneous["peaksAmplitude"]])

# Encode string labels into integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_sorted_homogeneous["Racket Type"])


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Racket Type
RB    188
RO    188
RR    188
RV    188
Name: count, dtype: int64


C:\Users\Abdelaziz\AppData\Local\Temp\ipykernel_4420\426371751.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min_samples_per_age, random_state=42))


In [7]:
from sklearn.ensemble import RandomForestClassifier

# Define the ranges for hyperparameters
n_estimators_range = range(1, 21)  # Number of estimators (trees)
criterion_options = ['gini', 'entropy', 'log_loss']  # Splitting criteria

# List to store the results
results = []

# Test all combinations of hyperparameters
for n_estimators in n_estimators_range:
    for criterion in criterion_options:
        # Create and train the Random Forest model
        rtf = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, random_state=42)
        rtf.fit(X_train, y_train)

        # Evaluate on the test set
        y_pred = rtf.predict(X_test)
        accuracy_test = accuracy_score(y_test, y_pred)

        # Evaluate on the training set
        y_train_pred = rtf.predict(X_train)
        accuracy_train = accuracy_score(y_train, y_train_pred)

        # Add the results to the list
        results.append({
            'n_estimators': n_estimators,
            'criterion': criterion,
            'accuracy_train': accuracy_train,
            'accuracy_test': accuracy_test
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

print(results_df)


    n_estimators criterion  accuracy_train  accuracy_test
0              1      gini        0.878536       0.675497
1              1   entropy        0.876872       0.649007
2              1  log_loss        0.876872       0.649007
3              2      gini        0.878536       0.675497
4              2   entropy        0.895175       0.642384
5              2  log_loss        0.895175       0.642384
6              3      gini        0.950083       0.708609
7              3   entropy        0.945092       0.682119
8              3  log_loss        0.945092       0.682119
9              4      gini        0.960067       0.768212
10             4   entropy        0.950083       0.754967
11             4  log_loss        0.950083       0.754967
12             5      gini        0.975042       0.788079
13             5   entropy        0.973378       0.788079
14             5  log_loss        0.973378       0.788079
15             6      gini        0.975042       0.774834
16            